In [15]:
import cv2
import numpy as np
import pandas as pd

In [16]:
algo_name = 'sift'

In [17]:
with open(f'{algo_name}_output.txt') as file:
    lines = file.readlines()

In [18]:
lines = list(map(lambda t: list(map(int, t[:-1].split(' ')))[::-1], lines))

In [19]:
from collections import Counter
counter = Counter(np.array(lines)[:, 0])

In [20]:
clear_lines = list(filter(lambda line: counter[line[0]] != 2, lines))

In [21]:
def f(lines, x0):
    flag = True
    y = None
    for line in lines:
        if x0 == line[0]:
            y = line[-1]
            lines.remove(line)
            flag = False
            break
    return flag, y

In [22]:
x0 = clear_lines[0][0]
track = [x0]
tracks = []
while True:
    flag, x0 = f(clear_lines, x0)
    if flag == False:
        track.append(x0)
    else:
        tracks.append(track)
        if len(clear_lines) != 0:
            x0 = clear_lines[0][0]
            track = [x0]
        else:
            break

In [23]:
qqq = []
for track in tracks:
    for e in track:
        qqq.append(e)
len(set(qqq)) == len(qqq)

False

In [24]:
d = {}
for index in range(1, len(tracks) + 1):
    for bbox_index in tracks[index - 1]:
        d[bbox_index] = index

In [25]:
df = pd.read_csv('data/yolov5s.csv')
base_path = "C:\\#Projects\\Lanit\\SS2022_ITS\\Project2\\dataset_20220504_1445_cut\\17371610\\"

In [26]:
import random
colors = []
for i in range(1000):
    colors.append([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)])

In [27]:
def in_road(row):
    left_line = lambda x: -0.4861111111111111 * x + 650.7777777777778
    right_line = lambda x: 0.40669240 * x - 166.49935649
    return (left_line(row['xmax']) <= row['ymax']) and (right_line(row['xmin']) <= row['ymin'])

In [28]:
from tqdm import tqdm
for i in tqdm(range(451, 600)):
    image_name = f'{i:010}.tiff'
    selected_df = df[df['frame'] == image_name]
    image = cv2.imread(base_path + image_name)
    for index in selected_df.index.values:
        if index in d:            
            row = selected_df.loc[index]
            color = colors[d[index]]
            if in_road(row):
                image = cv2.rectangle(image, (row['xmin'], row['ymin']), (row['xmax'], row['ymax']), color, 2)
                image = cv2.putText(image, 'track' + str(d[index]), (row['xmin'], row['ymin']), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1, cv2.LINE_AA)
    cv2.imwrite(f'tracks/{algo_name}/{i}.png', image)

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:41<00:00,  3.58it/s]


In [73]:
images_names = {f'{i:010}.tiff': (i - 450) for i in range(451, 501)}
df['index'] = df.index
short_df = df[np.array(list(map(lambda i: i in d.keys(), df['index'].values)))].copy()
short_df['track_id'] = df['index'].map(d)
short_df['track_id'] = short_df['track_id'].astype(int)
short_df['frame_index'] = short_df['frame'].map(images_names)
short_df = short_df.dropna()
short_df['frame_index'] = short_df['frame_index'].astype(int)

In [74]:
short_df['in_road'] = short_df.apply(in_road, axis=1)
short_df = short_df[short_df['in_road']].copy()

In [75]:
short_df['width'] = (short_df['xmax'] - short_df['xmin']).astype(int)
short_df['height'] = (short_df['ymax'] - short_df['ymin']).astype(int)
short_df['conf'] = -1
short_df['x'] = -1
short_df['y'] = -1
short_df['z'] = -1

In [76]:
short_df[['frame_index', 'track_id', 'xmin', 'ymin', 'width', 'height', 'conf', 'x', 'y', 'z']].to_csv(
    f'{algo_name}_tracks.txt', 
    sep=',',
    index=False,
    header=False
)

In [77]:
short_df[['frame_index', 'track_id', 'xmin', 'ymin', 'width', 'height', 'conf', 'x', 'y', 'z']]

,frame_index,track_id,xmin,ymin,width,height,conf,x,y,z
0,1,4,1,540,361,350,-1,-1,-1,-1
1,1,1,1385,549,171,111,-1,-1,-1,-1
2,1,2,561,378,129,138,-1,-1,-1,-1
7,2,1,1368,540,152,101,-1,-1,-1,-1
10,2,2,568,385,124,138,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
546,50,49,1172,723,213,177,-1,-1,-1,-1
547,50,48,1037,387,63,51,-1,-1,-1,-1
549,50,46,521,398,65,47,-1,-1,-1,-1
550,50,52,322,692,218,188,-1,-1,-1,-1
